In [41]:
import pandas as pd

#import salling data that is a pickle file 
df = pd.read_pickle('data/salling_data.pkl')

#import recipe data
df2 = pd.read_csv('data/recipes_data.csv')

In [42]:
#keep the only columns we need
df = df[['Store Name', 'Category 1','Cleaned Ingredients', 'Original Price', 'New Price', 'Discount','Percent Discount']]
df.head()

#print(df['Cleaned Ingredients'].unique())

,Store Name,Category 1,Cleaned Ingredients,Original Price,New Price,Discount,Percent Discount
1,Netto Lyngby,Hotdog Buns,hotdog bun,14.00,7.0,7.00,50.00
2,Netto Lyngby,Light Dark Buns,light dark bun,25.00,15.0,10.00,40.00
3,Netto Lyngby,Seeded Rye Breads,rye bread,7.75,6.0,1.75,22.58
4,Netto Lyngby,Buttermilk,buttermilk,13.25,10.0,3.25,24.53
5,Netto Lyngby,Other Pork,pork,59.95,36.0,23.95,39.95


In [43]:
df2.head(5)

,id,source,link,title,0
0,1,bbc,https://www.bbc.co.uk/food/recipes/healthy_chi...,,beef mince
1,1,bbc,https://www.bbc.co.uk/food/recipes/healthy_chi...,,red onion
2,1,bbc,https://www.bbc.co.uk/food/recipes/healthy_chi...,,garlic
3,1,bbc,https://www.bbc.co.uk/food/recipes/healthy_chi...,,courgette
4,1,bbc,https://www.bbc.co.uk/food/recipes/healthy_chi...,,aubergine


In [44]:
#how many recipes we have, count unique titles
print(f"there are {df2['id'].nunique()} unique recipes")

#how many products we have, count unique description_en
print(f"there are {df2['description_en'].nunique()} unique ingredients")


there are 1550 unique recipes


KeyError: 'description_en'

In [40]:
# Merge the dataframes on the 'Category 1' column from df and 'description_en' column from df2
merged_df = pd.merge(df2, df, left_on='description_en', right_on='Cleaned Ingredients', how='left')

# Display the merged dataframe
merged_df.head(5)

,id,link,title,title_en,description_en,source,Store Name,Category 1,Cleaned Ingredients,Original Price,New Price,Discount,Percent Discount
0,1,https://www.bbc.co.uk/food/recipes/slow_cooker...,Slow cooker pea and ham soup,Slow cooker pea and ham soup,sunflower oil,bbc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,https://www.bbc.co.uk/food/recipes/slow_cooker...,Slow cooker pea and ham soup,Slow cooker pea and ham soup,onion,bbc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,https://www.bbc.co.uk/food/recipes/slow_cooker...,Slow cooker pea and ham soup,Slow cooker pea and ham soup,celery,bbc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,https://www.bbc.co.uk/food/recipes/slow_cooker...,Slow cooker pea and ham soup,Slow cooker pea and ham soup,carrots,bbc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,https://www.bbc.co.uk/food/recipes/slow_cooker...,Slow cooker pea and ham soup,Slow cooker pea and ham soup,split peas,bbc,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Count unique recipes that have a match in the Salling data
print(f"There are {merged_df['id'].nunique()} unique recipes that have a match in the Salling data")

# Group by recipe title and count the unique matched ingredients
matched_counts = merged_df.dropna(subset=['Cleaned Ingredients']).groupby('title')['Cleaned Ingredients'].nunique().reset_index()
matched_counts.columns = ['id', 'matched_ingredients_count']

# Identify ingredients from the recipes that did not have a match
unmatched_ingredients = merged_df[merged_df['Cleaned Ingredients'].isna()]

# Count the number of unmatched ingredients for each recipe
unmatched_counts = unmatched_ingredients.groupby('title')['description_en'].nunique().reset_index()
unmatched_counts.columns = ['id', 'unmatched_ingredients_count']

# Merge the matched and unmatched counts into a single DataFrame
final_counts = pd.merge(matched_counts, unmatched_counts, on='title', how='outer').fillna(0)

# Add columns for matched and unmatched ingredients
final_counts['matched_ingredients'] = final_counts['title'].map(lambda x: merged_df[(merged_df['id'] == x) & (merged_df['Cleaned Ingredients'].notna())]['Cleaned Ingredients'].unique())
final_counts['unmatched_ingredients'] = final_counts['title'].map(lambda x: merged_df[(merged_df['id'] == x) & (merged_df['Cleaned Ingredients'].isna())]['description_en'].unique())

# Display the final DataFrame
final_counts = final_counts.sort_values(by='matched_ingredients_count', ascending=False)
final_counts


There are 462 unique recipes that have a match in the Salling data


,title,matched_ingredients_count,unmatched_ingredients_count,matched_ingredients,unmatched_ingredients
99,Gluten-free creamy chicken and mash pie,4.0,11,"[butter, milk, cheese, cream cheese]","[potatoes, wholegrain mustard, black pepper, g..."
232,The ultimate Christmas burger,4.0,28,"[butter, milk, bacon, sausage]","[turkey, thyme, rosemary, cinnamon, goose fat,..."
234,Toffee apple self-saucing pudding,4.0,11,"[butter, milk, apple juice, cream]","[caster sugar, Bramley apples, eggs, self-rais..."
219,Spiced roast chicken,3.0,23,"[chicken, butter, yoghurt]","[lemons, onions, orange, coconut milk, chicken..."
199,Roast harissa chicken with homemade flatbreads,3.0,15,"[butter, chicken, yoghurt]","[cumin, coriander seeds, smoked paprika, haris..."
...,...,...,...,...,...
322,Healthy spaghetti bolognese,0.0,12,[],"[olive oil, beef mince, onion, mushrooms, carr..."
323,Healthy stir-fry with tofu and rice noodles,0.0,14,[],"[garlic, soy sauce, peanut butter, ginger, chi..."
324,Homemade granola,0.0,12,[],"[maple syrup, caster sugar, sunflower oil, van..."
325,Honey and mustard chicken with rustic potatoes,0.0,9,[],"[chicken legs, new potatoes, olive oil, black ..."


In [ ]:
#extract final_counts to a csv file
final_counts.to_csv('data/final_counts.csv', index=False)